In [62]:
import pandas as pd
import xgboost as xgb
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [63]:
card = pd.read_csv("D:\\Downloads\\creditcard.csv\\creditcard.csv")

In [64]:
print(card.head())
print(card.tail())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [65]:
print(card["Class"].value_counts())

0    284315
1       492
Name: Class, dtype: int64


In [66]:
train,test = train_test_split(card, test_size = 0.3, random_state = 42)

In [67]:
class_0 = train[train['Class']==0]
class_1 = train[train['Class']==1]

In [68]:
undersampled_class_0 = resample(class_0,
                               replace = False,
                               n_samples = len(class_1),
                               random_state = 42)
undersampled_data = pd.concat([undersampled_class_0, class_1])
print(undersampled_data['Class'].value_counts())

0    356
1    356
Name: Class, dtype: int64


In [69]:
x_train = undersampled_data.drop(columns=['Class'])
y_train = undersampled_data['Class']
d_train = xgb.DMatrix(x_train, label = y_train)

In [70]:
x_test = test.drop(columns=['Class'])
y_test = test['Class']
d_test = xgb.DMatrix(x_test, label=y_test)

In [71]:
xgboost_mod = xgb.train({'objective':'binary:logistic','eval_metric':'error','eta':0.1},
                       d_train, num_boost_round = 100)

In [72]:
pred = xgboost_mod.predict(d_test)
binary_pred = [1 if  p > 0.5 else 0 for p in pred]

In [73]:
accuracy = accuracy_score(y_test, binary_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.9630279835679927


In [74]:
confusion_mat = confusion_matrix(y_test, binary_pred)
print("Confusion Matrix: \n", confusion_mat)

Confusion Matrix: 
 [[82156  3151]
 [    8   128]]
